# only SlogP descriptors

In [1]:
import pandas as pd
import numpy as np
import json

from tqdm import tqdm
from xgboost import XGBClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import cohen_kappa_score
from numpy import linspace

#from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils import class_weight
#from sklearn.experimental import enable_halving_search_cv
#from sklearn.model_selection import HalvingRandomSearchCV


In [2]:
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

import warnings
warnings.filterwarnings("ignore", category=UserWarning)


In [3]:
#test_run = True
test_run = False

In [4]:
df = pd.read_csv("Temp/df_proc_train_set_xgb.csv")

In [5]:
df['sol_category'] = pd.to_numeric(df['sol_category'], downcast='integer')
df

,Id,smiles,sol_category,ecfp_0,ecfp_1,ecfp_2,ecfp_3,ecfp_4,ecfp_5,ecfp_6,...,rdkit_desc_fr_quatN,rdkit_desc_fr_sulfide,rdkit_desc_fr_sulfonamd,rdkit_desc_fr_sulfone,rdkit_desc_fr_term_acetylene,rdkit_desc_fr_tetrazole,rdkit_desc_fr_thiazole,rdkit_desc_fr_thiophene,rdkit_desc_fr_unbrch_alkane,rdkit_desc_fr_urea
0,EOS12286,Cc1nc(N2CCN(C(=O)Nc3ccc(F)cc3F)CC2)cc(-n2ccnc2)n1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,EOS85869,CCN(CC)[C@H]1CCN(C(=O)Cc2nc(C(C)C)c(C)s2)C1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,EOS85435,CNC(=O)CNC(=O)c1c(-n2cccc2)sc(C)c1C,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,EOS102302,CC(C)(C)c1ccc(CSc2cnn(C(C)(C)C)c(=O)c2Cl)cc1,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,EOS64213,CC[C@H](NC(=O)c1ccnc(-n2ccnc2)c1)c1ccccc1OC,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70706,EOS37839,O=C(NCCCc1nc(=O)[nH][nH]1)[C@H]1CCC(F)(F)C1,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
70707,EOS2088,Cc1ccc(C(=O)NC2CCCC2)cc1S(=O)(=O)N1CCOCC1,2,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70708,EOS10587,COCCN1CCC(CN(C)S(=O)(=O)c2cccc(C(F)(F)F)c2)C1,2,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70709,EOS40533,O=C(Nc1ccc(F)cc1)NC1CCN(C(=O)Cc2cnn(-c3ccccc3)...,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [6]:
df['sol_category'].value_counts()

2    65835
1     2835
0     2041
Name: sol_category, dtype: int64

In [7]:
if test_run:
    df, _, __, ___ = train_test_split(df, df['sol_category'], train_size=500, stratify=df['sol_category'])

In [8]:
df['sol_category'].value_counts(normalize=True)

2    0.931043
1    0.040093
0    0.028864
Name: sol_category, dtype: float64

In [9]:
#y = pd.read_excel(xl, 'y', header = None)

In [10]:
# hypothesis fuction
# takes input variables and theta, return calculated output variable

In [11]:
print(len(df))
#print(len(y))
X = df.copy(deep=True)
X.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)
X

70711


,ecfp_0,ecfp_1,ecfp_2,ecfp_3,ecfp_4,ecfp_5,ecfp_6,ecfp_7,ecfp_8,ecfp_9,...,rdkit_desc_fr_quatN,rdkit_desc_fr_sulfide,rdkit_desc_fr_sulfonamd,rdkit_desc_fr_sulfone,rdkit_desc_fr_term_acetylene,rdkit_desc_fr_tetrazole,rdkit_desc_fr_thiazole,rdkit_desc_fr_thiophene,rdkit_desc_fr_unbrch_alkane,rdkit_desc_fr_urea
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70706,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
70707,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70708,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70709,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [12]:
y = df.iloc[:, 2]

In [13]:
y

0        0
1        0
2        0
3        0
4        0
        ..
70706    2
70707    2
70708    2
70709    2
70710    2
Name: sol_category, Length: 70711, dtype: int8

In [14]:
y.unique()

array([0, 1, 2], dtype=int8)

In [15]:
# Stricter feature selection
### 1. just use ecfp/rdkit+example
### 2. Hyperparatuning (StratifiedKfold+RandomizedSearchCV on X_train and y_train) (max_iter, solver, penalty,C)

# for dataset over all_dataset
#   hyperparameter tuning()

### restrict data set to SlogP descriptors

In [16]:
#SlogP = [col for col in X.columns if 'logP' in col]
X_SlogP = X[[col for col in X.columns if ('logP' in col or 'LogP' in col)]]
X_SlogP

,example_SlogP,example_XLogP,rdkit_desc_SlogP_VSA1,rdkit_desc_SlogP_VSA10,rdkit_desc_SlogP_VSA11,rdkit_desc_SlogP_VSA12,rdkit_desc_SlogP_VSA2,rdkit_desc_SlogP_VSA3,rdkit_desc_SlogP_VSA4,rdkit_desc_SlogP_VSA5,rdkit_desc_SlogP_VSA6,rdkit_desc_SlogP_VSA7,rdkit_desc_SlogP_VSA8,rdkit_desc_MolLogP
0,2.60302,2.599,10.216698,25.080616,0.000000,0.000000,56.629085,0.000000,18.558179,5.824404,42.986475,0.000000,5.817863,2.60302
1,3.06012,1.417,0.000000,0.000000,0.000000,11.336786,52.911844,11.215359,6.923737,55.612375,0.000000,0.000000,0.000000,3.06012
2,1.63144,0.645,10.633577,0.000000,0.000000,11.336786,29.973887,4.794537,13.847474,20.798587,24.526421,0.000000,5.001082,1.63144
3,5.24150,6.034,5.559267,0.000000,0.000000,23.362825,9.780485,16.706769,0.000000,52.669326,40.152333,5.022633,0.000000,5.24150
4,3.15700,3.222,10.053652,0.000000,5.749512,0.000000,27.552034,0.000000,0.000000,35.307840,61.316053,0.000000,5.817863,3.15700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70706,0.58220,1.272,11.006532,8.780830,0.000000,0.000000,33.555807,11.215359,5.917906,31.507691,4.794537,0.000000,0.000000,0.58220
70707,1.68832,1.600,5.316789,0.000000,0.000000,0.000000,50.975310,14.760154,6.923737,41.604727,23.094585,0.000000,0.000000,1.68832
70708,2.29420,2.078,0.000000,13.171245,0.000000,0.000000,64.566300,20.936453,5.917906,11.984273,29.160952,0.000000,0.000000,2.29420
70709,3.36660,2.887,10.633577,14.872339,0.000000,0.000000,45.750042,11.215359,5.817221,18.405095,66.990991,0.000000,5.687386,3.36660


In [17]:
X['rdkit_desc_MolLogP']

0        2.60302
1        3.06012
2        1.63144
3        5.24150
4        3.15700
          ...   
70706    0.58220
70707    1.68832
70708    2.29420
70709    3.36660
70710    0.52880
Name: rdkit_desc_MolLogP, Length: 70711, dtype: float64

In [18]:
# apply StandardScaler
scaler = StandardScaler(copy=True, with_mean=True, with_std=True).fit(X)
X = scaler.fit_transform(X)

# split dataset
StratifiedKFold(n_splits=5)
X_train, X_val, y_train, y_val = train_test_split(X_SlogP, y, random_state=1,  shuffle=True, stratify=y) #, test_size=0.2, train_size=0.8)

In [19]:
y_train.value_counts()

2    49376
1     2126
0     1531
Name: sol_category, dtype: int64

In [20]:
X_train.shape

(53033, 14)

In [21]:
X_train

,example_SlogP,example_XLogP,rdkit_desc_SlogP_VSA1,rdkit_desc_SlogP_VSA10,rdkit_desc_SlogP_VSA11,rdkit_desc_SlogP_VSA12,rdkit_desc_SlogP_VSA2,rdkit_desc_SlogP_VSA3,rdkit_desc_SlogP_VSA4,rdkit_desc_SlogP_VSA5,rdkit_desc_SlogP_VSA6,rdkit_desc_SlogP_VSA7,rdkit_desc_SlogP_VSA8,rdkit_desc_MolLogP
26059,2.35120,1.665,15.787319,5.687386,5.749512,0.000000,23.405220,4.794537,0.000000,10.357989,60.794147,0.000000,10.902925,2.35120
66007,0.02350,0.231,4.899910,5.817863,0.000000,0.000000,91.780039,0.000000,5.414990,32.228043,12.654640,0.000000,11.163878,0.02350
59939,3.50880,3.336,10.633577,4.794537,0.000000,0.000000,18.617712,0.000000,0.000000,64.208216,11.649125,0.000000,0.000000,3.50880
1399,1.99492,2.475,10.216698,5.817863,0.000000,0.000000,27.958818,17.760115,6.923737,29.662474,36.659155,0.000000,0.000000,1.99492
47792,3.36862,2.618,14.790515,0.000000,11.499024,0.000000,24.277607,0.000000,6.923737,60.106574,22.722196,0.000000,0.000000,3.36862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58500,0.14280,-0.587,5.138974,13.171245,0.000000,0.000000,18.615161,16.199590,0.000000,5.563451,17.289171,0.000000,0.000000,0.14280
37812,2.87962,2.832,11.006532,0.000000,0.000000,11.761885,33.005806,0.000000,6.923737,41.735203,35.256849,0.000000,5.687386,2.87962
47819,2.30650,2.382,0.000000,13.171245,0.000000,0.000000,42.994368,19.634269,0.000000,18.666048,30.853741,0.000000,0.000000,2.30650
8647,1.80700,2.039,21.509633,10.688468,0.000000,22.937726,59.218536,4.794537,0.000000,10.357989,52.769646,5.022633,5.687386,1.80700


In [22]:
y_train

26059    2
66007    2
59939    2
1399     0
47792    2
        ..
58500    2
37812    2
47819    2
8647     2
35459    2
Name: sol_category, Length: 53033, dtype: int8

In [23]:
y_train.value_counts()

2    49376
1     2126
0     1531
Name: sol_category, dtype: int64

In [24]:

# setup parameters for xgboost
params = {}
#params['booster'] = ['gbtree', 'gblinear', 'dart']
#params['objective'] = ['binary:logistic']
#params["eval_metric"] = ["error"]
params['eta'] = 0.0001 #, 0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5] # boosting learning rate
params['gamma'] = 0.4 #linspace(0.000000001, 1.0, num=11) # min loss red recuired for further partition on leaf node
params['max_depth'] = 5 #np.arange(1, 11, 2) # max tree dept for base learners
params['n_estimators'] = 250 #np.arange(50, 550, 50)
params['min_child_weight'] = 1 # min sum of instance weight in a child
params['max_delta_step'] = 0 # max delta step allowed for each tree's weight estimate
params['subsample']= 0.5 #[0.5, 1] # subsample ratio of training instance
params['colsample_bytree'] = 1 # subsample ratio of columns when cunstructing each tree
#params['silent'] = [1]
#params['seed'] = [0] # = random_state ???
params['base_score'] = 0.5 # initial prediction score, global bias
#params['random_state'] = [0] # = seed ???
#params['scale_pos_weight'] = ratio
params['n_jobs'] = 5

In [25]:
classes_weights = class_weight.compute_sample_weight(
    class_weight='balanced',
    y=y_train
    )

model = XGBClassifier(**params)

model.fit(X_train, y_train, sample_weight=classes_weights)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False, eta=0.0001,
              eval_metric=None, feature_types=None, gamma=0.4, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=9.99999975e-05,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=250,
              n_jobs=5, num_parallel_tree=1, objective='multi:softprob', ...)

In [26]:

valPredictions = model.predict(X_val)

#quad kappa score
sk_quad_kappa = cohen_kappa_score(y_val, valPredictions, weights='quadratic')
    
# printing results
print(f'quadratic kappa score of the validation data set: {sk_quad_kappa}')


quadratic kappa score of the validation data set: 0.04243204197086048


In [27]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_val, valPredictions, labels=[2, 1, 0])

array([[11543,  3253,  1663],
       [  420,   196,    93],
       [  305,   116,    89]])

In [28]:

real_test_set = pd.read_csv('Temp/df_proc_test_set_xgb.csv')
sub_template = pd.read_csv('Data/submission_template_rdm.csv')
(sub_template['Id'] == real_test_set['Id']).value_counts()


True    30307
Name: Id, dtype: int64

In [29]:

real_test_set.drop(columns=['Id', 'smiles'], inplace=True)
real_test_set = real_test_set[[col for col in X.columns if 'logP' in col]]

# apply StandardScaler
scaler = StandardScaler(copy=True, with_mean=True, with_std=True).fit(real_test_set)
real_test_set = scaler.fit_transform(real_test_set)

# prediction
realtestPredictions = model.predict(real_test_set)
sub_template['pred'] = realtestPredictions


AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [ ]:
set(realtestPredictions)

{0, 1, 2}

In [ ]:

sub_template['pred'].value_counts()


2    21167
1     5894
0     3246
Name: pred, dtype: int64

## save as csv

In [ ]:
#sub_template.to_csv('Submissions/submission_25_11_C-men.csv', index=False)